<a href="https://colab.research.google.com/github/stephenbeckr/numerical-analysis-class/blob/student/Demos/Ch2_AitkenExtrapolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aitken Extrapolation

## Task 1: recreate Table 2.10 in the book
In the table, we have $x_n = \cos(\frac{1}{n})$ which converges to $\cos(0)=1$ since $\cos$ is continuous.  Use Aitken Extrapolation to accelerate this sequence $(x_n)$.

You can do this with a `for` loop, or you can use Python slice notation to do it in a very compact fashion.

In [ ]:
# Recreate Table 2.10 in the book

import numpy as np
from numpy import diff as D   # this is the Delta in the book, aka "forward difference"

N = 7  # total number of terms
p = np.cos( 1/np.r_[1:N+1] ) # Example in book (Table 2.10)
print("The sequence p_n is:")
for pn in p:
  print("{:.5f}".format(pn))

print("The sequence \hat{p}_n is:")  
# Define phat (for p^hat, not as in "that is a phat song!")

phat = ... TODO ...

for pn in phat:
  print("{:.5f}".format(pn))

## Task 2: accelerate a root-finding problem
You could do Steffensen's variant, but we didn't discuss that, so I suggest you just to Aitken extrapolation

Let's solve $x = g(x)$ where
$$g(x) = .7\cos(x)$$
insdie the interval $[0,1]$.  We know that $g$ is a contraction since
$$|g'(x)| = |-.7\sin(x)| \le .7 < 1$$

In [3]:
# Try a root-finding problem, x = .7*cos(x)
from scipy.optimize import root_scalar
g = lambda x : .7*np.cos(x)

output = root_scalar(lambda x : g(x) - x ,bracket=[0,1])
trueRoot = output.root
print("The root is",trueRoot)

The root is 0.5839889663069239


In [ ]:
# Accelerate the iteration

# Try fixed-point iteration, starting at 0
x = 0
N   = 20
p   = np.zeros(N)
phat= np.zeros(N-2)
# There are many ways to program this, depending on how/if you want to save
#   all the iterates.
for i in range(N):
  x = g(x)
  p[i] = x
  
  ... TODO (accelerate the sequence) ...